# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [3]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**(2.5) - a**0.5

xs = [2,3,4]

print(f(*xs)) # f(2,3,4)

6.336362190988558


In [4]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  df_da = -3*a**2 - 0.5/a**(0.5)
  df_db = cos(3*b)*3 + 2.5*b**(1.5)
  df_dc = 1.0/c**2
  return [df_da, df_db, df_dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(*xs)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [10]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
h = 10e-7 # small enough step size to be a good approximation
numerical_grad = [0, 0, 0]
for i in range(len(xs)):
  xs_p = xs[::]
  xs_p[i] += h # x_i + h
  numerical_grad[i] = (f(*xs_p) - f(*xs)) / h # rise / run
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [13]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
h = 10e-4 # small enough step size to be a good approximation
numerical_grad2 = [0, 0, 0]
for i in range(len(xs)):
  xs_p = xs[::]
  xs_q = xs[::]
  xs_p[i] += h # x_i + h
  xs_q[i] -= h # x_i - h
  numerical_grad2[i] = (f(*xs_p) - f(*xs_q)) / (2*h) # rise / run
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353554401639766
OK for dim 1: expected 10.25699027111255, yours returns 10.256994551617105
OK for dim 2: expected 0.0625, yours returns 0.06250000390650712


## section 2: support for softmax

In [41]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = self.__class__(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  def __neg__(self): # -self
      return self * -1

  def __radd__(self, other):
    return self + other

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = self.__class__(self.data * other.data, (self, other), '*')

    def _backward():
      """
      dO/dself
      = dO/dout * dout/dself
      = out.grad * d(self.data * other.data)/dself
      = out.grad * other.data
      """
      self.grad += out.grad * other.data
      other.grad += out.grad * self.data

    out._backward = _backward

    return out

  def __pow__(self, other):
    assert other*0 == 0, "Passed value is not numerical"
    x = self.data
    t = x**other
    out = self.__class__(data = t, _children = (self,), label = f"**{other}")

    def _backward():
      """
      dO/dself
      = dO/dout * dout/dself
      = out.grad * d(x**a) / dself
      = out.grad * a*x**(a-1)      
      """
      self.grad += out.grad * other*(x)**(other-1)
  
    out._backward = _backward

    return out

  def __truediv__(self, other):
    return self*other**-1

  def exp(self):
    x = self.data
    t = exp(x)
    out = self.__class__(data=t, _children = (self,), label = "exp")

    def _backward():
      """
      dO/self
      = dO/dout * dout/dself
      = out.grad * d exp(x) / dself
      = out.grad * exp(x)
      """
      self.grad += out.grad * t

    out._backward = _backward

    return out
  
  def log(self):
    x = self.data
    t = log(x)
    out = self.__class__(data=t, _children=(self,), label='log')

    def _backward():
      """
      dOut/dself
      = dout/self * dout/dself
      = out.grad * d(log(x))/dself
      = out.grad * 1/x
      = out.grad * x**-1
      """
      self.grad = out.grad * x**-1

    out._backward = _backward

    return out
  # ------

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [42]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [48]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
import torch.autograd

xs = [0.0, 3.0, -2.0, 1.0]
tensor = torch.Tensor(xs)
tensor = torch.autograd.Variable(tensor)
activation = torch.nn.functional.log_softmax(tensor, dim = -1)
activation.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn